In [3]:
import numpy as np
import scipy
import cv2, random
from scipy import fftpack

from math import cos, sin
from numpy import zeros, ones, prod, array, pi, log, min, mod, arange, sum, mgrid, exp, pad, round
from numpy.random import randn, rand
from scipy.signal import convolve2d

"""
下面有三个函数生成模糊核：
1. make_kernels(psz, mxsz, nc, num)
此函数生成大小为 mxsz * mxsz 的num个运动模糊核，返回一个大小为mxsz * mxsz * num的数组，psz和nc属于生成模糊核时需要的参数。
该生成方式源于Ayan Chakrabarti 的A neural approach to blind motion deblurring，根据本论文的matlab代码改写

在陈亮的a_Non-Blind_Deblurring_Network_for_Night_Blurry_Images一文中也用到了此生成模糊核的方法


以下两个函数均参考USRNet
2. blurkernel_synthesis(h=37, w=None)
生成大小为h*w的运动模糊核，如果w不指定，生成h*h大小的模糊核

3. gen_kernel(k_size=np.array([25, 25]), scale_factor=np.array([4, 4]), min_var=0.6, max_var=12., noise_level=0)
生成大小为k_size的高斯模糊核
"""
#########################################################################################################
def matlabMax(a, b):
    mask, mask1 = a > b, b > a
    return a * mask + b * mask1

def matlabMin(a, b):
    mask, mask1 = a < b, b < a
    return a * mask + b * mask1

def matlabIndex(arr, ind, data):
    begin_index = 0
    l, w = arr.shape[0], arr.shape[1]
    for i in range(l):
        for j in range(w):
            if begin_index >= data.shape[0]:
                return 0
            if (i - 1) * l + j == ind[begin_index]:
                arr[j, i] = data[begin_index]
                begin_index += 1
            

def make_kernels(psz, mxsz, nc, num):
    K = np.zeros((mxsz, mxsz, num), dtype=float)

    imp = np.zeros((mxsz, mxsz))
    imp[int((mxsz + 1) / 2), int((mxsz + 1) / 2)] = 1

    xg, yg = np.meshgrid(np.array([np.arange(1,psz+1)]), np.array([np.arange(1,psz+1)]))

    for i in range(num):
        while True:
            x = np.random.randint(1, psz, size=(1, nc))
            y = np.random.randint(1, psz, size=(1, nc))
            
            x = scipy.interpolate.UnivariateSpline(np.linspace(0, 1, nc), x)(np.linspace(0, 1, nc * 5000))
            x = np.round(matlabMax(1, matlabMin(psz, x)))
            
            y = scipy.interpolate.UnivariateSpline(np.linspace(0, 1, nc), y)(np.linspace(0, 1, nc * 5000))
            y = np.round(matlabMax(1, matlabMin(psz, y)))

            idx = (x - 1) * psz + y
            idx = np.unique(idx)
            wt = matlabMax(0, np.random.randn(idx.shape[0], 1, 1) * 0.5 + 1)
            if np.sum(wt) == 0:
                continue
        
            wt = wt / np.sum(wt)

            krnl = np.zeros((psz, psz))
            matlabIndex(krnl, idx, wt)

            # print(krnl)
            cx = int(np.round(np.sum(np.multiply(krnl, xg))))
            cy = int(np.round(np.sum(np.multiply(krnl, yg))))
            if cx <= psz / 2:
                krnl = np.concatenate((np.zeros((psz, psz - 2 * cx + 1), dtype=np.float64), krnl), axis=1)
                # krnl = np.array([np.zeros((psz, psz - 2 * cx + 1))],[krnl])
            else:
                krnl = np.concatenate((krnl, np.zeros((psz, 2 * cx - psz - 1), dtype=np.float64)), axis=1)
                # krnl = np.array([krnl], [np.zeros((psz,2 * cx - psz - 1))])
            p2 = krnl.shape[1]
            if cy <= psz / 2:
                krnl = np.concatenate((np.zeros((psz - 2 * cy + 1, p2), dtype=float), krnl), axis=0)
                # krnl = np.array([[np.zeros((psz - 2 * cy + 1,p2))],[krnl]])
            else:
                krnl = np.concatenate((krnl, np.zeros((2 * cy - psz - 1, p2), dtype=float)), axis=0)
                # krnl = np.array([[krnl],[np.zeros((2 * cy - psz - 1,p2))]])
            if np.amax(krnl.shape) <= mxsz:
                break
        
        K[:, :, i] = scipy.signal.convolve2d(imp, krnl, 'same')
        return K

################################################################################################
def rot3D(x, r):
    Rx = array([[1, 0, 0], [0, cos(r[0]), -sin(r[0])], [0, sin(r[0]), cos(r[0])]])
    Ry = array([[cos(r[1]), 0, sin(r[1])], [0, 1, 0], [-sin(r[1]), 0, cos(r[1])]])
    Rz = array([[cos(r[2]), -sin(r[2]), 0], [sin(r[2]), cos(r[2]), 0], [0, 0, 1]])
    R = Rz @ Ry @ Rx
    x = R @ x
    return x

def fspecial_gauss(size, sigma):
    x, y = mgrid[-size // 2 + 1 : size // 2 + 1, -size // 2 + 1 : size // 2 + 1]
    g = exp(-((x ** 2 + y ** 2) / (2.0 * sigma ** 2)))
    return g / g.sum()

def randomTrajectory(T):
    x = zeros((3, T))
    v = randn(3, T)
    r = zeros((3, T))
    trv = 1 / 1
    trr = 2 * pi / T
    for t in range(1, T):
        F_rot = randn(3) / (t + 1) + r[:, t - 1]
        F_trans = randn(3) / (t + 1)
        r[:, t] = r[:, t - 1] + trr * F_rot
        v[:, t] = v[:, t - 1] + trv * F_trans
        st = v[:, t]
        st = rot3D(st, r[:, t])
        x[:, t] = x[:, t - 1] + st
    return x

def kernelFromTrajectory(x):
    h = 5 - log(rand()) / 0.15
    h = round(min([h, 27])).astype(int)
    h = h + 1 - h % 2
    w = h
    k = zeros((h, w))

    xmin = min(x[0])
    xmax = max(x[0])
    ymin = min(x[1])
    ymax = max(x[1])
    xthr = arange(xmin, xmax, (xmax - xmin) / w)
    ythr = arange(ymin, ymax, (ymax - ymin) / h)

    for i in range(1, xthr.size):
        for j in range(1, ythr.size):
            idx = (
                (x[0, :] >= xthr[i - 1])
                & (x[0, :] < xthr[i])
                & (x[1, :] >= ythr[j - 1])
                & (x[1, :] < ythr[j])
            )
            k[i - 1, j - 1] = sum(idx)
    if sum(k) == 0:
        return
    k = k / sum(k)
    k = convolve2d(k, fspecial_gauss(3, 1), "same")
    k = k / sum(k)
    return k

def fspecial_gaussian(hsize, sigma):
    hsize = [hsize, hsize]
    siz = [(hsize[0]-1.0)/2.0, (hsize[1]-1.0)/2.0]
    std = sigma
    [x, y] = np.meshgrid(np.arange(-siz[1], siz[1]+1), np.arange(-siz[0], siz[0]+1))
    arg = -(x*x + y*y)/(2*std*std)
    h = np.exp(arg)
    h[h < scipy.finfo(float).eps * h.max()] = 0
    sumh = h.sum()
    if sumh != 0:
        h = h/sumh
    return h

# USRNet生成运动模糊核
def blurkernel_synthesis(h=37, w=None):
    # https://github.com/tkkcc/prior/blob/879a0b6c117c810776d8cc6b63720bf29f7d0cc4/util/gen_kernel.py
    w = h if w is None else w
    kdims = [h, w]
    x = randomTrajectory(250)
    k = None
    while k is None:
        k = kernelFromTrajectory(x)

    # center pad to kdims
    pad_width = ((kdims[0] - k.shape[0]) // 2, (kdims[1] - k.shape[1]) // 2)
    pad_width = [(pad_width[0],), (pad_width[1],)]
    
    if pad_width[0][0]<0 or pad_width[1][0]<0:
        k = k[0:h, 0:h]
    else:
        k = pad(k, pad_width, "constant")
    x1,x2 = k.shape
    if np.random.randint(0, 4) == 1:
        k = cv2.resize(k, (random.randint(x1, 5*x1), random.randint(x2, 5*x2)), interpolation=cv2.INTER_LINEAR)
        y1, y2 = k.shape
        k = k[(y1-x1)//2: (y1-x1)//2+x1, (y2-x2)//2: (y2-x2)//2+x2]
        
    if sum(k)<0.1:
        k = fspecial_gaussian(h, 0.1+6*np.random.rand(1))
    k = k / sum(k)
    # import matplotlib.pyplot as plt
    # plt.imshow(k, interpolation="nearest", cmap="gray")
    # plt.show()
    return k

#######################################################################################################################
# 生成高斯模糊核
def gen_kernel(k_size=np.array([25, 25]), scale_factor=np.array([4, 4]), min_var=0.6, max_var=12., noise_level=0):
    """"
    # modified version of https://github.com/assafshocher/BlindSR_dataset_generator
    # Kai Zhang
    # min_var = 0.175 * sf  # variance of the gaussian kernel will be sampled between min_var and max_var
    # max_var = 2.5 * sf
    """
    sf = random.choice([1, 2, 3, 4])
    scale_factor = np.array([sf, sf])
    # Set random eigen-vals (lambdas) and angle (theta) for COV matrix
    lambda_1 = min_var + np.random.rand() * (max_var - min_var)
    lambda_2 = min_var + np.random.rand() * (max_var - min_var)
    theta = np.random.rand() * np.pi  # random theta
    noise = 0#-noise_level + np.random.rand(*k_size) * noise_level * 2

    # Set COV matrix using Lambdas and Theta
    LAMBDA = np.diag([lambda_1, lambda_2])
    Q = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta), np.cos(theta)]])
    SIGMA = Q @ LAMBDA @ Q.T
    INV_SIGMA = np.linalg.inv(SIGMA)[None, None, :, :]

    # Set expectation position (shifting kernel for aligned image)
    MU = k_size // 2 - 0.5*(scale_factor - 1) # - 0.5 * (scale_factor - k_size % 2)
    MU = MU[None, None, :, None]

    # Create meshgrid for Gaussian
    [X,Y] = np.meshgrid(range(k_size[0]), range(k_size[1]))
    Z = np.stack([X, Y], 2)[:, :, :, None]

    # Calcualte Gaussian for every pixel of the kernel
    ZZ = Z-MU
    ZZ_t = ZZ.transpose(0,1,3,2)
    raw_kernel = np.exp(-0.5 * np.squeeze(ZZ_t @ INV_SIGMA @ ZZ)) * (1 + noise)

    # shift the kernel so it will be centered
    #raw_kernel_centered = kernel_shift(raw_kernel, scale_factor)

    # Normalize the kernel and return
    #kernel = raw_kernel_centered / np.sum(raw_kernel_centered)
    kernel = raw_kernel / np.sum(raw_kernel)
    return kernel



    
# k1 = make_kernels(21, 21, 6, 16)
# k = gen_kernel()
from tqdm import tqdm
for i in tqdm(range(100)):
    k1 = blurkernel_synthesis(h=21)
    while np.count_nonzero(k1[:1]) or np.count_nonzero(k1[-1:]) or np.count_nonzero(k1[:, :1]) or np.count_nonzero(k1[:, -1:]):
        k1 = blurkernel_synthesis(h=21)
    k1 = k1 / k1.max()
    k1 = k1 * 255
    k1 = k1.round()
    cv2.imwrite('.\\move_psf\\%07d.png'%i, k1)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:02<00:00, 42.01it/s]
